In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [12]:
df = pd.read_csv('Материал_для_ТЗ_дата_аналитик_задание_1.csv', dtype=str, keep_default_na=False)

In [13]:
df

,date
0,20.9.1901
1,6.5.1999
2,31.7.1988
3,5.8.1982
4,18.1.1992
...,...
1999995,16.2.1992
1999996,12.11.1994
1999997,9.10
1999998,


In [14]:
df.isnull().sum()

,0
date,0


In [15]:
df.describe()

,date
count,2000000
unique,32550
top,
freq,291132


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   date    object
dtypes: object(1)
memory usage: 15.3+ MB


In [18]:
# добавляем год 1900 там, где в дате не указан год
mask_no_year = df['date'].str.count('\.') == 1
df.loc[mask_no_year, 'date'] = df.loc[mask_no_year, 'date'] + '.1900'

# парсим все даты через pandas (непарсящиеся станут NaT)
dates_parsed = pd.to_datetime(df['date'], format='%d.%m.%Y', errors='coerce', dayfirst=True)

# форматируем даты в строковый вид DD-MM-YYYY
df['date'] = dates_parsed.dt.strftime('%d-%m-%Y')
df['date'] = df['date'].fillna('')


In [19]:
# Сохраняем, порядок записей сохранён
df.to_csv('normalized_dates.csv', index=False)


In [21]:
norm_df = pd.read_csv('normalized_dates.csv')
norm_df

,date
0,20-09-1901
1,06-05-1999
2,31-07-1988
3,05-08-1982
4,18-01-1992
...,...
1999995,16-02-1992
1999996,12-11-1994
1999997,09-10-1900
1999998,NaN


In [22]:
from datetime import datetime

# функция для нормализации одной даты
def normalize_date(date_str):
    if date_str is None:
        return ''
    date_str = date_str.strip()
    if date_str == '':
        return ''
    # если года нет, дописываем 1900
    if date_str.count('.') == 1:
        date_str = date_str + '.1900'
    try:
        dt = datetime.strptime(date_str, '%d.%m.%Y')
    except ValueError:
        return ''
    return dt.strftime('%d-%m-%Y')

# тесы
assert normalize_date('6.5.1999') == '06-05-1999'
assert normalize_date('9.7') == '09-07-1900'
assert normalize_date('') == ''
assert normalize_date(None) == ''
assert normalize_date('29.2.1900') == '' # 1900 не високосный – дата не валидна
assert normalize_date('-14.10') == '' # некорректный формат
print('работает!')


работает!


In [27]:
report = {
    'total_rows' : len(df),
    'parsed_ok'  : df.notna().sum(),
    'added_1900' : mask_no_year.sum(),
    'invalid_rows': df.isna().sum()
}
print(report)
print(df.loc[mask_no_year].head(10))

{'total_rows': 2000000, 'parsed_ok': date    2000000
dtype: int64, 'added_1900': np.int64(259823), 'invalid_rows': date    0
dtype: int64}
          date
9   09-07-1900
19  23-10-1900
21  08-08-1900
22  11-05-1900
25  21-06-1900
29  31-08-1900
37  26-05-1900
47  16-06-1900
64  18-12-1900
85  10-12-1900
